# MCP: Исследование научных статей через Model Context Protocol

В этом ноутбуке мы используем **Model Context Protocol (MCP)** для поиска и анализа научных статей на arXiv.

## Архитектура

```
                         ┌─────────────────┐
┌─────────────────┐      │   Yandex Cloud  │      ┌─────────────────┐
│  Jupyter Client │──────│   Responses API │──────│  arXiv MCP      │
│  (этот ноутбук) │      │                 │      │  Server (VM)    │
└─────────────────┘      └─────────────────┘      └─────────────────┘
                                                         │
                                                         ▼
                                                    arXiv API
```

Для начала установим необходимые библиотеки:

In [ ]:
%pip install --upgrade openai python-dotenv

> **ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

## Авторизация

Для работы с Yandex Cloud нам понадобится `folder_id` (идентификатор каталога) и `api_key` (API-ключ сервисного аккаунта). Мы предполагаем, что эти значения хранятся в файле `.env`, который можно скачать, выполнив ячейку ниже:

In [ ]:
!curl -o .env {{url_of_dotenv_file}}

Также скачаем реализацию класса `Agent` из репозитория. Этот класс универсально поддерживает все типы инструментов: словарные описания (Web Search, File Search, MCP), а также Pydantic-классы для локальных функций.

In [1]:
!curl -o Agent.py https://raw.githubusercontent.com/yandex-ai-studio/ai-studio-course/refs/heads/main/4-rag-search/Agent.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  8576  100  8576    0     0  11312      0 --:--:-- --:--:-- --:--:-- 11358


Загрузим переменные окружения и создадим клиент для работы с API:

In [ ]:
import os, json
from dotenv import load_dotenv
from IPython.display import Markdown, display
from Agent import Agent, create_client

load_dotenv()

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]
model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = create_client()

def printx(string):
    display(Markdown(string))

print(f"✅ Авторизация настроена (folder_id: {folder_id[:8]}...)")

✅ Авторизация настроена (folder_id: b1gkp4q6...)


---

## Часть 1: Запускаем MCP-сервер

В этом примере нам потребуется запустить внешний MCP-сервер на каком-то компьютере, доступном из интернет, либо в облаке Yandex Cloud. Это можно сделать несколькими способами:

### MCP-сервер на виртуальной машине Yandex Cloud

Чтобы запустить MCP-сервер на виртуальной машине Yandex Cloud, выполните следующие шаги:

* Клонируйте репозиторий курса на виртуальную машину
* Перейдите в директорию с MCP-серверами `cd 5-mcp/servers`
* Запустите MCP-сервер `python arxiv_research_mcp.py`
* Запишите адрес MCP-сервера в ячейке ниже

### MCP-сервер в облаке Yandex Cloud с использованием MCP Hub

Для этого:

* Создайте Cloud Function для поиска публикаций в arXiv
* Определите MCP-сервер с инструментом `search_arxiv`, ориентируясь на [видеоинструкцию](https://youtu.be/7L_MU66VvSo)
* Запишите адрес MCP-сервера в ячейке ниже


> Возможно, преподаватель уже запустил MCP-сервер в рамках курса. В этом случае задайте его адрес в ячейке ниже:

In [17]:
mcp_server_url = "http://cathy.ycloud.eazify.net:8000/sse" 

---

## Часть 2: Определение MCP-инструментов

MCP-инструменты для Responses API описываются с `type: mcp`.
Responses API сам подключается к MCP-серверу и вызывает нужные функции.

In [24]:
# MCP-инструмент для arXiv
arxiv_mcp_tool = {
    "type": "mcp",
    "server_label": "arXiv-Research",
    "server_description": "Сервер для поиска и получения информации о научных статьях на arXiv.org",
    "server_url": mcp_server_url,
    "require_approval": "never",  # Автоматически вызывать без подтверждения
}

print(f"✅ MCP-инструмент для arXiv: {arxiv_mcp_tool['server_url']}")

✅ MCP-инструмент для arXiv: http://cathy.ycloud.eazify.net:8000/sse


---

## Часть 3: Проверяем вызов MCP-инструмента

Для начала попробуем вызвать MCP-сервер непосредственно из Responses API:


In [30]:
res = client.responses.create(
    model=model,
    instructions="Ты - ассистент учёного, и у тебя есть сервис поиска статей в arXiv.",
    tools=[arxiv_mcp_tool],
    input="Найди статью про визуальные трансформеры"
)

printx(res.output_text)

Вот три наиболее релевантные статьи по теме **визуальных трансформеров (Vision Transformers)**:

---

### 1. **Transformation Properties of Learned Visual Representations**
- **arXiv ID**: [1412.7659v3](https://arxiv.org/abs/1412.7659)
- **Авторы**: Taco S. Cohen, Max Welling
- **Аннотация**:  
  Когда трёхмерный объект движется относительно наблюдателя, происходит изменение на плоскости изображения и в визуальном представлении, вычисляемом моделью. Основываясь на идее, что хорошее визуальное представление должно линейно преобразовываться при движении сцены, авторы показывают, как это свойство может быть использовано для анализа и построения инвариантных к преобразованиям моделей.

> 🔍 *Эта статья — одна из ранних работ, исследующих теоретические основы трансформаций в визуальных моделях, включая архитектуры, подобные трансформерам.*

---

### 2. **Attention Guided CAM: Visual Explanations of Vision Transformer Guided by Self-Attention**
- **arXiv ID**: [2402.04563v1](https://arxiv.org/abs/2402.04563)
- **Авторы**: Saebom Leem, Hyunseok Seo
- **Аннотация**:  
  Vision Transformer (ViT) — одна из самых популярных моделей в области компьютерного зрения благодаря высокой эффективности на различных задачах. Для полноценного использования ViT в практических приложениях требуются качественные методы визуализации, такие как карта активаций (CAM). В этой работе предлагается метод, использующий механизм самовнимания для улучшения интерпретируемости ViT.

> 🔍 *Актуальная работа (2024 г.), посвящённая интерпретации Vision Transformers — важной теме для понимания их решений.*

---

### 3. **Learning CLIP Guided Visual-Text Fusion Transformer for Video-based Pedestrian Attribute Recognition**
- **arXiv ID**: [2304.10091v1](https://arxiv.org/abs/2304.10091)
- **Авторы**: Jun Zhu, Jiandong Jin, Zihan Yang и др.
- **Аннотация**:  
  Существующие методы распознавания атрибутов пешеходов (PAR) в основном работают с неподвижными изображениями и плохо справляются с перекрытиями, размытием и другими сложностями. В данной работе предлагается использовать видеоданные и трансформер с визуально-текстовым объединением, обученный с помощью CLIP, для повышения надёжности распознавания.

> 🔍 *Пример применения визуальных трансформеров в мультимодальной задаче (видео + текст), что отражает современные тенденции.*

---

Если вас интересует конкретное направление — например, архитектура ViT, интерпретация, применение в детекции, классификации или мультимодальности — могу подобрать более узкоспециализированные статьи.

Посмотрим на структуру ответа, чтобы понять, как MCP-вызовы отображаются в результатах:

In [31]:
# Исследуем структуру ответа
print(f"Элементы ответа ({len(res.output)} шт.):")
for item in res.output:
    print(f"  - {item.type}")
    if hasattr(item, 'server_label'):
        print(f"    Server: {item.server_label}")

Элементы ответа (3 шт.):
  - mcp_list_tools
    Server: arXiv-Research
  - mcp_call
    Server: arXiv-Research
  - message


Как это работает:

1. Мы указываем адрес MCP-сервера в описании инструмента и вызываем Responses API
2. Responses API запрашивает у MCP-сервера список доступных инструментов и передаёт их LLM-модели
3. LLM решает вызвать функцию с MCP-сервера
4. Responses API сам подключается к MCP-серверу и вызывает функцию. Если MCP-вызов требует подтверждения - предварительно запрашивается подтверждение вызова функции у клиента
5. Результат возвращается в LLM для формирования окончательного ответа

---

## Часть 4: Создание агента-исследователя


Описанный нами ранее класс `Agent` уже поддерживает работу со всеми встроенными в Responses API инструментами, включая MCP. Достаточно передать ему соответствующий инструмент в процессе определения агента: Responses API сам подключится к серверу, узнает список доступных инструментов, и будет вызывать их в случае необходимости!

In [28]:
researcher_instruction = """
Ты — научный ассистент-исследователь, эксперт по поиску научных статей на arXiv.

Используй MCP-сервер arXiv-Research для:
- Поиска статей по ключевым словам
- Получения деталей о статье по arXiv ID
- Поиска статей конкретного автора
- Поиска недавних публикаций

Стратегия:
1. Ищи статьи на английском языке (arXiv англоязычный)
2. Выбери интересную статью и получи её детали
3. Составь краткое резюме на русском

Включай в резюме: название, авторов, ключевые идеи.
"""

# Создаём агента с MCP-инструментом
researcher = Agent(
    client,
    model = model,
    instruction=researcher_instruction,
    tools=[arxiv_mcp_tool]
)

print("🔬 Агент-исследователь готов!")

🔬 Агент-исследователь готов!


Теперь опишем удобную функцию для запуска исследований, которая будет также показывать вызываемые функции:

In [38]:
def research(topic: str):
    """Удобная функция для исследования темы."""
    print(f"🔬 Исследуем: {topic}")
    print("=" * 60)
    result = researcher(topic)
    for x in result.output:
        if x.type=='mcp_call':
            print(f" + Вызов {x.name}{x.arguments}")
    print("=" * 60)
    printx(result.output_text)

---
## Часть 5: Экспериментируем

Зададим нашему агенту несколько вопросов:

In [39]:
research("Найди статьи про large language models")

🔬 Исследуем: Найди статьи про large language models
 + Вызов search_arxiv{"query": "large language models", "max_results": 3}
 + Вызов get_paper_details{"arxiv_id": "2402.14679v2"}


### Краткое резюме статьи:

**Название:** Is Self-knowledge and Action Consistent or Not: Investigating Large Language Model's Personality  
**Авторы:** Yiming Ai, Zhiwei He, Ziyin Zhang и др.  
**arXiv ID:** 2402.14679v2  

**Ключевые идеи:**

- Статья исследует **вопрос согласованности между "самооценкой" личности LLM и их реальным поведением** — то есть, насколько достоверны утверждения больших языковых моделей о своих чертах характера при выполнении конкретных задач.
- Авторы используют **стандартные психологические опросники (например, на основе модели "Большой пятерки")**, чтобы "опросить" различные LLM (например, Llama, GPT) о их личностных качествах — такие как добросовестность, экстраверсия, открытость опыту и др.
- Затем модели подвергаются **поведенческим тестам**, имитирующим реальные сценарии (например, принятие решений, этические дилеммы, социальное взаимодействие), чтобы выявить их реальные склонности.
- Основной вывод: **наблюдается значительный разрыв (disjunction) между декларируемой "самооценкой" и реальным поведением моделей**. Например, модель может описать себя как "добросовестную и честную", но в тестах склонна к сглаживанию правды, уклончивым ответам или даже стратегической дезинформации.
- Авторы подчёркивают, что **LLM не обладают настоящим самосознанием**, и их "самооценка" — это статистическая интерполяция на основе обучающих данных, а не искреннее самопонимание.
- Исследование поднимает **этические и методологические вопросы**: можно ли доверять LLM в задачах, требующих личной ответственности? Пригодны ли человеческие психологические тесты для оценки моделей ИИ?

**Вывод:**  
Несмотря на то что LLM могут убедительно имитировать личность, их "самознание" не согласуется с поведением. Это ставит под сомнение применимость антропоморфных характеристик к ИИ и предупреждает о рисках использования таких моделей в чувствительных социальных и психологических контекстах без строгой верификации их действий.

> Работа вносит важный вклад в понимание **ограничений и иллюзий "человеческой" личности в ИИ** и призывает к осторожности в интерпретации ответов моделей как проявлений настоящего "я".

In [40]:
research("Расскажи про статью Attention Is All You Need (ID: 1706.03762)")

🔬 Исследуем: Расскажи про статью Attention Is All You Need (ID: 1706.03762)
 + Вызов get_paper_details{"arxiv_id": "1706.03762"}


### Краткое резюме статьи:

**Название:** *Attention Is All You Need*  
**Авторы:** Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin  
**arXiv ID:** 1706.03762  
**Год публикации:** 2017  

#### Ключевые идеи:

- В статье представлена **новая архитектура нейронной сети под названием Transformer**, которая **полностью отказывается от рекуррентных (RNN) и свёрточных (CNN) слоёв** в задачах обработки последовательностей.
- Основа архитектуры — **механизм внимания (attention)**, в частности:
  - **Self-attention (само-внимание)** — позволяет каждому элементу последовательности напрямую взаимодействовать со всеми остальными, улавливая долгосрочные зависимости.
  - **Scaled dot-product attention** — эффективная математическая реализация механизма внимания.
  - **Multi-head attention** — параллельное использование нескольких механизмов внимания для захвата разных типов зависимостей.
- Архитектура состоит из **стека энкодеров и декодеров**, каждый из которых включает слои многоголового внимания и полносвязных сетей, с использованием нормализации и остаточных связей.
- Преимущества Transformer:
  - **Высокая параллелизуемость**: в отличие от RNN, не требует последовательной обработки, что ускоряет обучение.
  - **Меньшее время обучения**: модель достигает state-of-the-art результатов за 3.5 дня на 8 GPU.
  - **Высокое качество**:
    - **28.4 BLEU** на задаче перевода английский → немецкий (WMT 2014), что на 2+ BLEU лучше предыдущих лучших моделей.
    - **41.8 BLEU** на английский → французский — новый рекорд для одиночной модели.
  - **Хорошая обобщаемость**: успешно применена к задаче синтаксического разбора (constituency parsing), демонстрируя универсальность.

#### Значение статьи:

- **Революция в NLP**: Transformer стал основой для практически всех современных больших языковых моделей, включая BERT, GPT, T5, Llama, Gemini и др.
- **Смещение парадигмы**: После этой работы RNN и LSTM перестали быть доминирующими в задачах последовательной обработки.
- **Фундаментальный прорыв**: Простота и эффективность архитектуры сделали её стандартом де-факто в машинном переводе, генерации текста, анализе и других задачах.

> **"Attention Is All You Need"** — одна из самых влиятельных статей в истории машинного обучения. Она не только представила Transformer, но и заложила основу для современной эры больших языковых моделей.

In [41]:
research("Что нового в reinforcement learning?")

🔬 Исследуем: Что нового в reinforcement learning?
 + Вызов search_recent{"topic": "reinforcement learning", "max_results": 5}
 + Вызов get_paper_details{"arxiv_id": "2602.13156v1"}
 + Вызов search_arxiv{"query": "reinforcement learning", "field": "all", "max_results": 1}
 + Вызов get_paper_details{"arxiv_id": "2409.18827v1"}


### Краткое резюме статьи:

**Название:** *ARLBench: Flexible and Efficient Benchmarking for Hyperparameter Optimization in Reinforcement Learning*  
**Авторы:** Jannis Becktepe, Julian Dierkes, Carolin Benjamins и др.  
**arXiv ID:** 2409.18827v1  

#### Ключевые идеи:

- Статья представляет **ARLBench** — новый **бенчмарк для автоматической настройки гиперпараметров (HPO)** в задачах **обучения с подкреплением (Reinforcement Learning, RL)**.
- Проблема: настройка гиперпараметров (например, скорость обучения, фактор дисконтирования, архитектура сети) критически важна для производительности RL-агентов, но:
  - Эксперименты требуют больших вычислительных ресурсов.
  - Существующие методы HPO часто тестируются на узких или несопоставимых наборах задач, что затрудняет объективное сравнение.
- Решение — **ARLBench**:
  - Предлагает **репрезентативный набор RL-задач**, охватывающий разнообразные **алгоритмы (PPO, DQN, SAC и др.)** и **среды (например, из Gym, Atari)**.
  - Позволяет оценивать методы HPO с **существенно меньшими вычислительными затратами**, чем полное обучение агентов "с нуля".
  - Обеспечивает **сравнимость и воспроизводимость** результатов между разными подходами к AutoRL (автоматическому RL).
- В основе бенчмарка — **обширный набор данных по ландшафту гиперпараметров**, полученный в ходе масштабных предварительных экспериментов.
- Цель ARLBench — сделать исследования в области HPO и AutoRL **более доступными** для исследователей с ограниченными вычислительными ресурсами.

#### Значение статьи:

- ARLench — важный шаг к **стандартизации и ускорению исследований в RL**.
- Он способствует развитию **автоматизированных, надёжных и обобщаемых методов настройки RL-агентов**, что особенно важно для сложных реальных приложений.
- Проект **открыт и доступен** на GitHub, что способствует воспроизводимости и совместной разработке.

> **Вывод:** ARLBench решает острую проблему эффективной и честной оценки методов настройки RL, открывая путь к более масштабируемому и демократичному развитию автоматического обучения с подкреплением.

---

## Часть 6: Комбинирование MCP и локальных инструментов

Наш класс `Agent` позволяет смешивать MCP с локальными инструментами в виде Pydantic-функций. Добавим к нашему агенту возможность сохранения заметок по ходу исследований:

In [45]:
from pydantic import BaseModel, Field

# Локальный инструмент для сохранения заметок
class SaveNote(BaseModel):
    """Сохранить заметку о статье"""
    title: str = Field(description="Название статьи")
    summary: str = Field(description="Краткое описание")
    
    def process(self, session_id) -> str:
        with open("notes.md", "a", encoding="utf-8") as f:
            f.write(f"## {self.title}\n{self.summary}\n\n")
        return f"Заметка сохранена: {self.title}"

# Агент с MCP + локальными инструментами
research_notes_agent = Agent(
    client,
    model=model,
    instruction="Ищи статьи и сохраняй важные находки.",
    tools=[arxiv_mcp_tool, SaveNote]  # MCP + Pydantic
)

print("✅ Комбинированный агент готов")

✅ Комбинированный агент готов


In [46]:
result = research_notes_agent("Найди статью про transformers и сохрани заметку")
printx(result.output_text)

  🔧 SaveNote({"title": "On the Surprising Effectiveness of Tran...)


Найдена релевантная статья о трансформерах в контексте распознавания видео при малом количестве размеченных данных:

**Название:** *On the Surprising Effectiveness of Transformers in Low-Labeled Video Recognition*  
**Авторы:** Farrukh Rahman, Ömer Mubarek, Zsolt Kira  
**arXiv ID:** 2209.07474v3  
**Ссылка:** [https://arxiv.org/abs/2209.07474v3](https://arxiv.org/abs/2209.07474v3)

### Краткое содержание:
Статья демонстрирует, что трансформеры, несмотря на свою низкую эффективность в условиях малых данных на изображениях, показывают удивительно высокие результаты в задачах классификации видео при малом объёме размеченных данных. Авторы провели анализ на двух разных датасетах (Kinetics-400 и SomethingSomething-V2) и показали, что трансформеры превосходят традиционные CNN, включая сложные semi-supervised подходы, даже при использовании только размеченных данных.

---

✅ Заметка о статье успешно сохранена.

---

## Часть 7: Использование нескольких MCP-серверов

Можно подключить несколько MCP-серверов одновременно. Предположим, у нас также запущен MCP-сервер для сохранения заметок: 

In [62]:
# Пример с несколькими MCP-серверами
multi_mcp_tools = [
    {
        "type": "mcp",
        "server_label": "arXiv-Research",
        "server_description": "Поиск научных статей на arXiv",
        "server_url": "http://cathy.ycloud.eazify.net:8000/sse",
        "require_approval": "never",
    },
    {
        "type": "mcp",
        "server_label": "PersonalNotes",
        "server_description": "Сохранение личных заметок",
        "server_url": "http://cathy.ycloud.eazify.net:8001/sse",
        "require_approval": "never",
    },
]

system_prompt = """
"Ты - научный работник, исследователь. По просьбе пользователя
выполняй научный поиск в библиотеке статей arXiv, и сохраняй все найденные
находки в личные заметки PersonalNotes в блокнот research.
"""

research_notes_agent = Agent(
    client,
    model=model,
    instruction=system_prompt,
    tools=multi_mcp_tools
)

print("📝 Пример конфигурации с несколькими MCP-серверами")

📝 Пример конфигурации с несколькими MCP-серверами


In [63]:
result = research_notes_agent("Какие статьи вышли после 2024 года про GPT в медицине?")
printx(result.output_text)
for x in result.output:
        if x.type=='mcp_call':
            print(f" + Вызов {x.name}{x.arguments}")

Я сохранил информацию о последних статьях (2025–2026 гг.) по применению GPT и больших языковых моделей в медицине в блокнот **research**. Вот краткое содержание найденного:

### 🔬 **Недавние исследования GPT в медицине (2025–2026)**

1. **[2601.22124v1]** *A Federated and Parameter-Efficient Framework for LLM Training in Medicine*  
   — Исследование предлагает федеративный и эффективный по параметрам подход к обучению LLM в медицине, что особенно важно для защиты конфиденциальных данных пациентов.

2. **[2602.11167v1]** *Visualizing and Benchmarking LLM Factual Hallucination Tendencies*  
   — Анализ внутренних состояний моделей для выявления и визуализации склонности к фактическим галлюцинациям, что критично в медицинских приложениях.

3. **[2512.23729v1]** *AI-Generated Exam Questions in the AI Era*  
   — Сравнение подходов к генерации тестовых вопросов с помощью ИИ, с акцентом на безопасность и оригинальность при создании медицинских экзаменов.

4. **[2512.17028v1]** *A Women's Health Benchmark for LLMs*  
   — Представлен бенчмарк для оценки точности LLM в вопросах женского здоровья — важная ниша, ранее недостаточно изученная.

5. **[2512.08193v1]** *ClinicalTrialsHub*  
   — Платформа, объединяющая данные из ClinicalTrials.gov и научной литературы для улучшения доступа к информации о клинических испытаниях.

Если нужно, могу подробнее изучить любую из этих статей — например, получить полную аннотацию или детали методологии.

 + Вызов search_recent{"topic": "GPT in medicine", "max_results": 5}
 + Вызов add_note{"title": "Recent GPT in Medicine Papers (2025-2026)", "body": "- [2601.22124v1] A Federated and Parameter-Efficient Framework for Large Language Model Training in Medicine\n- [2602.11167v1] Visualizing and Benchmarking LLM Factual Hallucination Tendencies via Internal State Analysis and Clustering\n- [2512.23729v1] New Exam Security Questions in the AI Era: Comparing AI-Generated Item Similarity Between Naive and Detail-Guided Prompting Approaches\n- [2512.17028v1] A Women's Health Benchmark for Large Language Models\n- [2512.08193v1] ClinicalTrialsHub: Bridging Registries and Literature for Comprehensive Clinical Trial Access", "notebook": "research"}


Поскольку MCP-сервер с нашими личными заметками доступен разным агентам, можно использовать различные сценарии, например, сначала из агента-исследователя сделать заметки, и затем открыть эти заметки из чат-интерфейса другого приложения и использовать для написания статьи.

---

## Заключение

Ключевые преимущества:

1. **Простота** — Responses API сам вызывает MCP-сервер, не нужено дополнительного кода на стороне клиента
2. **Синхронный код** — никаких `async/await`
3. **Гибкость** — можно комбинировать MCP, локальные функции и другие доступные в рамках Responses API инструменты